In [2]:
# Active l'affichage de résultats multiples par cellule
# %matplotlib inline (pour des graphes)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from pandas import options
options.display.float_format = '{:.2f}'.format

NB_DECIMALES = 2

In [4]:
situation_18yo_moving_away = {
    "id": "18yo-moving-away",
    "description": "Un jeune de 18 ans déménage pour des études à l'université",
    "answers": {
        "statut-professionnel": "etudiant",  # activite
        "situation-professionnelle": "sans-emploi",  # dispatchSituationProfessionnelle
        "etudiant-mobilite": "parcoursup-nouvelle-region",  # dispatchEtudiantMobilite
        "boursier": True,  # boursier
        "date-naissance": "2007-03-01",  # date_naissance
        "handicap": False,  # handicap
        "statut-marital": "celibataire",  # statut_marital
        "code-postal-nouvelle-ville": "75101",  # depcom
        "situation-logement": "locataire",  # dispatchSituationLogement
        "type-logement": "logement-meuble",  # dispatchTypeLogement
        "logement-conventionne": True,  # logement_conventionne
        "colocation": False,  # coloc
        "logement-parente-proprietaire": False,  # proprietaire_proche_famille
        "nombre-personnes-logement": 1,  # 🔥 exclude: True
        "loyer-montant-mensuel": 700,  # loyer
        "loyer-montant-charges": 100,  # charges_locatives
        "loyer-difficile-payer": True,  # exclude: True
        "type-revenus": [
            "aucun-autres-revenus"
        ],
        "confirmation-end": [
            "confirmation-end-oui"
        ]
    },
    "questionsToApi": [
        "locapass-eligibilite",
        "mobilite-master-1",
        "mobilite-parcoursup",
        "aide-personnalisee-logement",
        "garantie-visale-eligibilite",
        "garantie-visale"
    ],
    "results": {
        "locapass": 1200,
        "locapass-eligibilite": True,
        "mobilite-master-1": 0,
        "mobilite-master-1-eligibilite": False,
        "mobilite-parcoursup": 500,
        "mobilite-parcoursup-eligibilite": True,
        "aide-personnalisee-logement": 327,
        "aide-personnalisee-logement-eligibilite": True,
        "garantie-visale": 800,
        "garantie-visale-eligibilite": True
    }
}

to_test = situation_18yo_moving_away

In [5]:
from notebooks.utils_mapping_simulateur import format_to_openfisca_json
from datetime import datetime
from dateutil.relativedelta import relativedelta


today = datetime.now()
today_year_month = today.strftime("%Y-%m")
year = today.strftime("%Y")
period = today_year_month  # par exemple : '2025-04'

last_month = today - relativedelta(months=1)
period_last_month  = last_month.strftime("%Y-%m")

situation_apl = format_to_openfisca_json(to_test, period)

In [6]:
from openfisca_core.simulation_builder import SimulationBuilder
from openfisca_france import FranceTaxBenefitSystem


tax_benefit_system = FranceTaxBenefitSystem()

sb = SimulationBuilder()
simulation = sb.build_from_entities(tax_benefit_system, situation_apl)

apl_variables = ['apl', 'logement_conventionne', 'aide_logement_montant', 'aide_logement_montant_brut', 'zone_apl']
# apl_variables_entities = ['familles', 'menages', 'familles', 'familles', 'menages']
# apl_variables_groupes = ['famille_1', 'menage_1', 'famille_1', 'famille_1', 'menage_1']

for variable_name in apl_variables: 
    print(f"{variable_name}: {simulation.calculate(variable_name, period)}")

parametres_secteur_locatif_period = tax_benefit_system.parameters(period).prestations_sociales.aides_logement.allocations_logement.locatif

apl: [181.65]
logement_conventionne: [ True]
aide_logement_montant: [181.65]
aide_logement_montant_brut: [236.66]
zone_apl: ['zone_1']


In [7]:
al_nb_personnes_a_charge = simulation.calculate('al_nb_personnes_a_charge', period)[0]
al_nb_personnes_a_charge

0

In [8]:
en_couple = simulation.calculate('en_couple', period)[0]
en_couple

celibataire = ~en_couple
celibataire

False

True

### Loyer pris en compte

```

L est le loyer mensuel réel pris en compte dans la limite d’un plafond variable en fonction de trois zones géographiques et du nombre de personnes à charge.

```

In [28]:
from openfisca_france.model.prestations.aides_logement import TypesZoneApl, TypesStatutOccupationLogement


zone_apl = simulation.calculate('zone_apl', period)[0]
id_zone_apl_applicable = TypesZoneApl.names[zone_apl]
print(f"Quelle est la zone APL du logement ? '{id_zone_apl_applicable}'")

loyer = simulation.calculate('loyer', period)[0]
aide_logement_loyer_reel = simulation.calculate('aide_logement_loyer_reel', period)[0]
L = aide_logement_loyer_reel

print(f"\nDans le cas d'un logement en chambre, d'une location meublée, d'une colocation ou d'une sous-location, le loyer L pris en compte est réduit.")

logement_chambre = simulation.calculate('logement_chambre', period)[0]
print(f"→ Le logement est-il une chambre ? {logement_chambre}")

statut_occupation_logement = simulation.calculate('statut_occupation_logement', period)[0]
location_meuble = statut_occupation_logement == TypesStatutOccupationLogement.locataire_meuble
print(f"→ Le logement est-il une location de meublé ? {location_meuble}")

coloc = simulation.calculate('coloc', period)[0]
print(f"→ Le logement est-il une colocation ? {coloc}")

print(f"→ Le logement est-il une sous-location ? [X] TODO")

print(f"\nPour un loyer déclaré de {loyer} €, le montant pris en compte ici est :\nL = {L} €")


Quelle est la zone APL du logement ? 'zone_1'

Dans le cas d'un logement en chambre, d'une location meublée, d'une colocation ou d'une sous-location, le loyer L pris en compte est réduit.
→ Le logement est-il une chambre ? False
→ Le logement est-il une location de meublé ? False
→ Le logement est-il une colocation ? False
→ Le logement est-il une sous-location ? [X] TODO

Pour un loyer déclaré de 700.0 €, le montant pris en compte ici est :
L = 700.0 €


#### Montant en euros du plafond de loyer L si logement chambre (quelle que soit la situation familiale)

In [ ]:
print(f"Le logement est-il une chambre ? {logement_chambre}")  # rappel
# if logement_chambre:



Le logement est-il une chambre ? False


#### Montant du plafond de loyer L selon situation familiale (hors logement chambre)

In [11]:
from pandas import DataFrame


id_zone_1 = TypesZoneApl.names[1]
id_zone_2 = TypesZoneApl.names[2]
id_zone_3 = TypesZoneApl.names[3]

parametres_locatif_zone_1 = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_1]
parametres_locatif_zone_2 = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_2]
parametres_locatif_zone_3 = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_3]


In [12]:
data_parametres_sans_personne_a_charge = {
    'Zone': [id_zone_1, id_zone_2, id_zone_3],
    'Personne seule': [parametres_locatif_zone_1.personnes_seules, parametres_locatif_zone_2.personnes_seules, parametres_locatif_zone_3.personnes_seules],
    'Couple': [parametres_locatif_zone_1.couples, parametres_locatif_zone_2.couples, parametres_locatif_zone_3.couples]
}

df_parametres_sans_personne_a_charge = DataFrame(data_parametres_sans_personne_a_charge)

# Définir la colonne 'Zone' comme index (optionnel, pour correspondre à l'exemple de la brochure APL)
df_parametres_sans_personne_a_charge = df_parametres_sans_personne_a_charge.set_index('Zone')

In [ ]:
from pandas import DataFrame, Index
from utils_display import colorie_reference


def get_plafond_loyer(zone_parameters, nb_personnes_a_charge):
    premier_enfant = zone_parameters.un_enfant if nb_personnes_a_charge > 0 else 0
    return premier_enfant + (zone_parameters.majoration_par_enf_supp * max(0, nb_personnes_a_charge - 1))

data_parametres_avec_personne_a_charge = {
    ('Personne seule ou couple', '1'): [parametres_locatif_zone_1.un_enfant, parametres_locatif_zone_2.un_enfant, parametres_locatif_zone_3.un_enfant],
    ('Personne seule ou couple', '2'): [get_plafond_loyer(parametres_locatif_zone_1, 2), get_plafond_loyer(parametres_locatif_zone_2, 2), get_plafond_loyer(parametres_locatif_zone_3, 2)],
    ('Personne seule ou couple', '3'): [get_plafond_loyer(parametres_locatif_zone_1, 3), get_plafond_loyer(parametres_locatif_zone_2, 3), get_plafond_loyer(parametres_locatif_zone_3, 3)],
    ('Personne seule ou couple', '4'): [get_plafond_loyer(parametres_locatif_zone_1, 4), get_plafond_loyer(parametres_locatif_zone_2, 4), get_plafond_loyer(parametres_locatif_zone_3, 4)],
    ('Personne seule ou couple', '5'): [get_plafond_loyer(parametres_locatif_zone_1, 5), get_plafond_loyer(parametres_locatif_zone_2, 5), get_plafond_loyer(parametres_locatif_zone_3, 5)],
    ('', 'Par pàc sup.'): [
        f'+ {parametres_locatif_zone_1.majoration_par_enf_supp}',
        f'+ {parametres_locatif_zone_2.majoration_par_enf_supp}',
        f'+ {parametres_locatif_zone_3.majoration_par_enf_supp}'
        ]
}

index = Index([id_zone_1, id_zone_2, id_zone_3], name='Zone')
df_parametres_avec_personne_a_charge = DataFrame(data_parametres_avec_personne_a_charge, index=index)

# Réorganiser les colonnes pour correspondre à l'ordre souhaité
df_parametres_avec_personne_a_charge = df_parametres_avec_personne_a_charge[[('Personne seule ou couple', '1'), ('Personne seule ou couple', '2'),
         ('Personne seule ou couple', '3'), ('Personne seule ou couple', '4'),
         ('Personne seule ou couple', '5'), ('', 'Par pàc sup.')]]

df_parametres_avec_personne_a_charge.columns.names = [None, 'Nombre de personnes à charge (pàc)']

Personne seule ou couple                \
Nombre de personnes à charge (pàc)                        1      2      3   
Zone                                                                        
zone_1                                               449.43 514.64 579.85   
zone_2                                               395.77 453.38 510.99   
zone_3                                               366.07 418.54 471.01   

                                                               
Nombre de personnes à charge (pàc)      4      5 Par pàc sup.  
Zone                                                           
zone_1                             645.06 710.27      + 65.21  
zone_2                             568.60 626.21      + 57.61  
zone_3                             523.48 575.95      + 52.47

In [14]:
from utils_calculate import get_latest_parameter_references

if (~logement_chambre):
    au_moins_une_personne_a_charge = al_nb_personnes_a_charge > 0
    print(f"Le ménage a-t-il des personnes à charge ? {au_moins_une_personne_a_charge}")


    plafond_loyer_period_zone = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_apl_applicable]
    if au_moins_une_personne_a_charge:
        plafond_loyer_applicable = get_plafond_loyer(plafond_loyer_period_zone, al_nb_personnes_a_charge)
        if (al_nb_personnes_a_charge > 5):
            print(f"Le ménage a plus de 5 personnes à charge : {al_nb_personnes_a_charge}")
        print(f"En zone '{id_zone_apl_applicable}', le plafond de loyer qui lui est applicable est : {plafond_loyer_applicable} €")
        df_parametres_avec_personne_a_charge.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=plafond_loyer_applicable)
    else:
        plafond_loyer_applicable = plafond_loyer_period_zone['personnes_seules' if celibataire else 'couples']
        print(f"En zone '{id_zone_apl_applicable}', le plafond de loyer qui lui est applicable est : {plafond_loyer_applicable} €")
        df_parametres_sans_personne_a_charge.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=plafond_loyer_applicable)

    print("Quelle est la référence législative à l'origine de ces valeurs ?")
    parameter_all_periods = tax_benefit_system.parameters.prestations_sociales.aides_logement.allocations_logement.locatif.formule.l_plafonds_loyers.par_zone.children[id_zone_apl_applicable].un_enfant
    get_latest_parameter_references(parameter_all_periods)


Le ménage a-t-il des personnes à charge ? False
En zone 'zone_1', le plafond de loyer qui lui est applicable est : 329.71 €


,Personne seule,Couple
Zone,,
zone_1,329.71,397.64
zone_2,287.35,351.72
zone_3,269.32,326.48


Quelle est la référence législative à l'origine de ces valeurs ?


[{'title': 'Arrêté du 27/09/2024, art. 1',
  'href': 'https://www.legifrance.gouv.fr/jorf/article_jo/JORFARTI000050278404'},
 {'title': 'Arrêté du 27/09/2019, art. 7',
  'href': 'https://www.legifrance.gouv.fr/loda/id/JORFTEXT000039160329/2024-09-29/'}]